Grad-CAM applied to MedViT model for binary classification

In [54]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
import torchvision.utils as vutils
import torchvision.transforms.functional as TF
from PIL import Image 
from MedViT_model import MedViT_base

In [55]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [56]:
# === Function to recreate and load trained MedViT model ===
def load_medvit_model(weight_path, num_classes=2, device=torch.device("cpu")):
    # Cria modelo com a mesma estrutura usada no treino
    model = MedViT_base(pretrained=False, num_classes=1000) #create inference model in the original format

    # Carrega pesos do modelo treinado
    state_dict = torch.load(weight_path, map_location=device)


    in_features = model.proj_head[0].in_features
    model.proj_head = torch.nn.Sequential(torch.nn.Linear(in_features, num_classes)) # Ajusta a camada de saída para o número correto de classes
    model.load_state_dict(state_dict, strict=False) # Load the state dict with strict=False to ignore the last layer
    model.to(device)
    model.eval()
    return model


In [57]:
# === Grad-CAM for MedViT ===
class GradCAM:
    def __init__(self, model, target_layer_name):
        self.model = model
        #self.target_layer = None
        self.gradients = None
        self.activations = None
        self._register_hooks(target_layer_name)

    def _register_hooks(self, target_layer_name):
        for name, module in self.model.named_modules():
            if name == target_layer_name:
                self.target_layer = module

                def forward_hook(module, input, output):
                    print("[Forward hook called]")
                    print("→ output shape:", output.shape)
                    self.activations = output

                def backward_hook(module, grad_input, grad_output):
                    print("[Backward hook called]")
                    print("→ grad_output[0] shape:", grad_output[0].shape)
                    self.gradients = grad_output[0]

                module.register_forward_hook(forward_hook)
                module.register_full_backward_hook(backward_hook)
                break
        if self.target_layer is None:
            raise ValueError(f"Layer {target_layer_name} not found in model")

    def generate(self, input_tensor, class_indices=None):
        print(f"[Grad-CAM] Input tensor shape before forward: {input_tensor.shape}")
        output = self.model(input_tensor)
        if class_indices is None: # if target class is not specified, it uses class prediction (argmax)
            class_indices = torch.argmax(output, dim=1)

        self.model.zero_grad() # Clear any previous gradients
        one_hot = torch.zeros_like(output, device=device)
        for i, class_idx in enumerate(class_indices):
            one_hot[i, class_idx] = 1

        output.backward(gradient=one_hot) #
        #self.model.zero_grad() # Ensure no gradients persist after Grad-CAM computation

        weights = torch.nn.functional.adaptive_avg_pool2d(self.gradients, 1)  # avgpool - reduces spatial resolution for output_size=1 -> [1, 512, 1, 1]
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True)
        cam = torch.relu(cam)

        cam = cam / (cam.max(dim=2, keepdim=True)[0].max(dim=3, keepdim=True)[0] + 1e-6)
        return cam.cpu().detach().numpy(), class_indices.cpu().numpy()

In [58]:
# Overlay heatmap
def overlay_heatmap(original_image, heatmap):
    # Debugging: Print heatmap shape and type
    print(f"[Debug] Initial heatmap shape: {heatmap.shape}, dtype: {heatmap.dtype}")

    # Handle cases where the heatmap has an extra dimension (1, H, W)
    if len(heatmap.shape) == 3 and heatmap.shape[0] == 1:
        heatmap = np.squeeze(heatmap, axis=0)
        print(f"[Debug] Squeezed heatmap shape: {heatmap.shape}")

    # Normalize the heatmap to range [0, 1] and convert to uint8
    heatmap = np.clip(heatmap, 0, 1)
    heatmap_uint8 = np.uint8(255 * heatmap)

    # Resize the heatmap to match the original image size
    heatmap_resized = cv2.resize(heatmap_uint8, (original_image.shape[1], original_image.shape[0]), interpolation=cv2.INTER_CUBIC)

    # Ensure the heatmap is in a compatible format for OpenCV
    if len(heatmap_resized.shape) == 2:  # Grayscale heatmap
        heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)
    elif len(heatmap_resized.shape) == 3 and heatmap_resized.shape[2] == 1:  # Single channel but 3D
        heatmap_color = cv2.applyColorMap(heatmap_resized[:, :, 0], cv2.COLORMAP_JET)
    else:
        print(f"[Error] Unexpected heatmap shape after resizing: {heatmap_resized.shape}")
        raise ValueError("Heatmap shape is not compatible with cv2.applyColorMap.")

    # Convert grayscale original image to BGR if necessary
    if len(original_image.shape) == 2:
        original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)

    # Blend the heatmap with the original image
    overlay = cv2.addWeighted(heatmap_color, 0.4, original_image, 0.6, 0)
    return overlay


In [59]:
# === Prediction Frame ===
def add_colored_frame(image, color, thickness=20):
    height, width = image.shape[:2]
    cv2.rectangle(image, (0, 0), (width - 1, height - 1), color, thickness)
    return image

In [60]:
# === Extract Ground Truth from folder path ===
def get_ground_truth(img_name, folder):
    # Combine the folder path and image name for a comprehensive check
    full_path = os.path.join(folder, img_name)
    
    # Check for the ground truth in a case-insensitive way
    if "cesarean" in full_path.lower():
        return "Cesarean Birth"
    elif "vaginal" in full_path.lower():
        return "Vaginal Birth"
    else:
        print(f"Warning: Could not determine ground truth for {full_path}. Assuming 'Cesarean Birth'.")
        ground_truth = "Cesarean Birth"

    # Print the ground truth for each image
    print(f"[Ground Truth] Image: {img_name} | Path: {full_path} | Picked: {ground_truth}")
    return ground_truth

In [61]:
#def save_raw_heatmap(heatmap, filename):
    # Normalize the heatmap to [0, 255]
    #heatmap -= heatmap.min()
    #heatmap /= (heatmap.max() + 1e-8)  # Prevent division by zero
    #heatmap_uint8 = np.uint8(255 * heatmap)  # Convert to 8-bit format
    
    # Apply colormap for visualization (optional)
    #heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    
    # Save the heatmap
    #cv2.imwrite(filename, heatmap_colored)
    #print(f"Raw heatmap saved at: {filename}")

In [ ]:
def apply_gradcam_and_save(model, input_folder, output_folder, grid_image_size=(224, 224)):
    os.makedirs(output_folder, exist_ok=True)
    grad_cam = GradCAM(model, 'features.28')

    transform = transforms.Compose([
        transforms.Resize((224, 224)), # Correct input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize for 1 channel 
    ])

    original_images = []
    correct_heatmaps = []
    incorrect_heatmaps = []
    misclassified_tensors = []
    correctly_classified_tensors = []

    class_names = ["Cesarean Birth", "Vaginal Birth"]

    for img_name in os.listdir(input_folder):
        if not img_name.lower().endswith((".png")):
            continue

        img_path = os.path.join(input_folder, img_name)
        image = Image.open(img_path).convert("RGB") 

        
        input_tensor = transform(image).unsqueeze(0).to(device)
        # Save the resized image as original (224x224) to ensure overlay consistency
        resized_image = transforms.Resize((224, 224))(image)
        original_np = np.array(resized_image)
        original_images.append(original_np)


        # Forward e GradCAM
        cam_heatmaps, predicted_classes = grad_cam.generate(input_tensor)
        prediction = predicted_classes[0]
        predicted_label = class_names[prediction]


        # === Ground truth from folder name ===
        ground_truth_label = get_ground_truth(img_name, input_folder)
        ground_truth_class = class_names.index(ground_truth_label)

        # Add frame
        frame_color = (0, 255, 0)  # green
        if prediction != ground_truth_class:
            frame_color = (0, 0, 255)  # red

        # === Overlay and save ===
        overlay = overlay_heatmap(original_np, cam_heatmaps[0]) # cam shape = [1,7,7]
        overlay_with_frame = add_colored_frame(overlay, frame_color, thickness=13)

        # === Save heatmap image ===
        save_path = os.path.join(output_folder, f"heatmap_{predicted_label}_{img_name}")
        cv2.imwrite(save_path, overlay_with_frame)
        print(f"Saved: {save_path} ; Predicted: {predicted_label}")

        # === Save heatmap for mean and tensor for grid ===
        overlay_rgb = cv2.cvtColor(overlay_with_frame, cv2.COLOR_BGR2RGB) #converts to rgb
        overlay_pil = Image.fromarray(overlay_rgb).resize(grid_image_size) # downsizes the image to fit the grid
        tensor_for_grid = TF.to_tensor(overlay_pil) #converts into a pytorch tensor [3,H,W] and normalizes pixel values

        if prediction == ground_truth_class:
            correctly_classified_tensors.append(tensor_for_grid)
            correct_heatmaps.append(cam_heatmaps[0][0])  # 2D spatial heatmap , cam shape = [7,7]
        else:
            misclassified_tensors.append(tensor_for_grid)
            incorrect_heatmaps.append(cam_heatmaps[0][0])


    # === CREATE GRID ===
    final_sorted_grid = misclassified_tensors + correctly_classified_tensors
    if final_sorted_grid:
        grid = vutils.make_grid(final_sorted_grid, nrow=20, padding=5, normalize=True)
        grid_path = os.path.join(output_folder, "grid_overlay_ordered.jpg")
        vutils.save_image(grid, grid_path)
        print(f"Grid saved at: {grid_path}")

    # === CALCULATE CORRECT MEAN HEATMAP ===
    if correct_heatmaps: #list of NumPy arrays (raw grad-cam maps before resizing or coloring), each w\ shape [3, 3]
        mean_heatmap = np.mean(np.stack(correct_heatmaps), axis=0) # turns list of N heatmaps into 3D array w\ shape [N, 7, 7]
        mean_heatmap -= mean_heatmap.min() # subtracts the minimum value from every pixel so the lowest value is 0
        mean_heatmap /= (mean_heatmap.max() + 1e-8) # divides every pixel by the new maximum value (normalize) ; adds 1e-8 to ensure it wont be divided by 0
        heatmap_uint8 = np.uint8(255 * mean_heatmap) # Converts [0.0 – 1.0] → [0 – 255] format needed for OpenCV
        heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET) # assigns color to the values

        # Resize to match original image size
        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_NEAREST) #INTER_CUBIC for smoother transitions ; INTER_NEAREST for blocky look
        heatmap_resized_smooth = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_CUBIC)


        mean_path_blocky = os.path.join(output_folder, "correct_mean_heatmap_blocky.png")
        cv2.imwrite(mean_path_blocky, heatmap_resized_blocky)
        print(f"Mean heatmap (correct only) saved at: {mean_path_blocky}")

        mean_path_smooth = os.path.join(output_folder, "correct_mean_smooth.png")
        cv2.imwrite(mean_path_smooth, heatmap_resized_smooth)
        print(f"Mean heatmap (correct only) saved at: {mean_path_smooth}")

    # === CALCULATE INCORRECT MEAN HEATMAP ===
    if incorrect_heatmaps:
        mean_heatmap_incorrect = np.mean(np.stack(incorrect_heatmaps), axis=0)
        mean_heatmap_incorrect -= mean_heatmap_incorrect.min()
        mean_heatmap_incorrect /= (mean_heatmap_incorrect.max() + 1e-8)
        heatmap_uint8_incorrect = np.uint8(255 * mean_heatmap_incorrect)
        heatmap_colored_incorrect = cv2.applyColorMap(heatmap_uint8_incorrect, cv2.COLORMAP_JET)

        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_NEAREST)
        heatmap_resized_smooth_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_CUBIC)

        mean_path_blocky_incorrect = os.path.join(output_folder, "incorrect_mean_heatmap_blocky.png")
        cv2.imwrite(mean_path_blocky_incorrect, heatmap_resized_blocky_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_blocky_incorrect}")

        mean_path_smooth_incorrect = os.path.join(output_folder, "incorrect_mean_smooth.png")
        cv2.imwrite(mean_path_smooth_incorrect, heatmap_resized_smooth_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_smooth_incorrect}")


    return correct_heatmaps, original_images

In [63]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_1/Abdomen_/train/Cesarean Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_cesarean_train_cv1_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/abdomen_cv1_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_cesarean_train_cv1_medvit_pretrained\heatmap_Vaginal Birth_PU10002107_abdomen1_276.png ; Predicted: Vaginal Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_p

In [64]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_1/Abdomen_/test/Cesarean Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_cesarean_test_cv1_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/abdomen_cv1_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_cesarean_test_cv1_medvit_pretrained\heatmap_Cesarean Birth_PU1005485_abdomen1_7.png ; Predicted: Cesarean Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pre

In [65]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_1/Abdomen_/train/Vaginal Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_vaginal_train_cv1_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/abdomen_cv1_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_vaginal_train_cv1_medvit_pretrained\heatmap_Cesarean Birth_PU10004004_abdomen1_0.png ; Predicted: Cesarean Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pr

In [66]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_1/Abdomen_/test/Vaginal Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_vaginal_test_cv1_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/abdomen_cv1_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_pretrained/X_224x224_ABDOMEN_vaginal_test_cv1_medvit_pretrained\heatmap_Cesarean Birth_PU10000998_abdomen1_201.png ; Predicted: Cesarean Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv1_MEDVIT_p

In [67]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Femur_/train/Cesarean Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_cesarean_train_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/femur_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_cesarean_train_cv3_medvit_pretrained\heatmap_Vaginal Birth_PU1003212_femur1_349.png ; Predicted: Vaginal Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretra

In [68]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Femur_/test/Cesarean Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_cesarean_test_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/femur_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_cesarean_test_cv3_medvit_pretrained\heatmap_Vaginal Birth_PU10002107_femur1_346.png ; Predicted: Vaginal Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretra

In [69]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Femur_/test/Vaginal Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_vaginal_test_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/femur_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_vaginal_test_cv3_medvit_pretrained\heatmap_Vaginal Birth_PU10012069_femur1_884.png ; Predicted: Vaginal Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrai

In [70]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Femur_/train/Vaginal Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_vaginal_train_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/femur_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_FEMUR_vaginal_train_cv3_medvit_pretrained\heatmap_Vaginal Birth_PU10000998_femur1_878.png ; Predicted: Vaginal Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretra

In [71]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Head_/train/Cesarean Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_cesarean_train_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/head_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_cesarean_train_cv3_medvit_pretrained\heatmap_Cesarean Birth_PU1003212_head1_201.png ; Predicted: Cesarean Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretra

In [72]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Head_/test/Cesarean Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_cesarean_test_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/head_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_cesarean_test_cv3_medvit_pretrained\heatmap_Cesarean Birth_PU10002107_head1_199.png ; Predicted: Cesarean Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretra

In [73]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Head_/test/Vaginal Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_vaginal_test_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/head_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_vaginal_test_cv3_medvit_pretrained\heatmap_Vaginal Birth_PU10012069_head1_585.png ; Predicted: Vaginal Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretraine

In [74]:
if __name__ == "__main__":
    input_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_3/Head_/train/Vaginal Birth"  # Folder with images to explain
    output_folder = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_vaginal_train_cv3_medvit_pretrained"   # Where to save heatmaps
    weight_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/MedViT_paths/head_cv3_medvit_pretrained_best-model.pth"  

    model = load_medvit_model(weight_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

initialize_weights...


C:\Users\anale\AppData\Local\Temp\ipykernel_9636\2192010331.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrained/X_224x224_HEAD_vaginal_train_cv3_medvit_pretrained\heatmap_Vaginal Birth_PU10000998_head1_579.png ; Predicted: Vaginal Birth
[Grad-CAM] Input tensor shape before forward: torch.Size([1, 3, 224, 224])
[Forward hook called]
→ output shape: torch.Size([1, 768, 7, 7])
[Backward hook called]
→ grad_output[0] shape: torch.Size([1, 768, 7, 7])
[Debug] Initial heatmap shape: (1, 7, 7), dtype: float32
[Debug] Squeezed heatmap shape: (7, 7)
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/224x224_MEDVIT/results_224x224_GradCAM_cv3_MEDVIT_pretrain